In [1]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [2]:
data = pd.read_csv('./gossipTimeSeries.csv', header=0, index_col=0)
gossip = pd.DataFrame(data)

gossip.head(10)

,_time,_measurement,identifier,experiment
0,2023-08-25 17:04:39.559375+00:00,iwant,1,152
1,2023-08-25 17:04:39.559955+00:00,iwant,1,152
2,2023-08-25 17:04:39.560356+00:00,iwant,1,152
3,2023-08-25 17:04:39.574611+00:00,iwant,1,152
4,2023-08-25 17:04:39.575129+00:00,iwant,1,152
5,2023-08-25 17:04:39.575320+00:00,iwant,1,152
6,2023-08-25 17:04:39.575510+00:00,iwant,1,152
7,2023-08-25 17:04:39.585263+00:00,iwant,1,152
8,2023-08-25 17:04:39.585524+00:00,iwant,1,152
9,2023-08-25 17:04:39.585600+00:00,iwant,1,152


In [3]:
gossip['type'] = 16
gossip.loc[gossip['_measurement'] == 'ihave', 'type'] = 17
gossip = gossip.drop(columns=['_measurement'])

gossip.head(10)

,_time,identifier,experiment,type
0,2023-08-25 17:04:39.559375+00:00,1,152,16
1,2023-08-25 17:04:39.559955+00:00,1,152,16
2,2023-08-25 17:04:39.560356+00:00,1,152,16
3,2023-08-25 17:04:39.574611+00:00,1,152,16
4,2023-08-25 17:04:39.575129+00:00,1,152,16
5,2023-08-25 17:04:39.575320+00:00,1,152,16
6,2023-08-25 17:04:39.575510+00:00,1,152,16
7,2023-08-25 17:04:39.585263+00:00,1,152,16
8,2023-08-25 17:04:39.585524+00:00,1,152,16
9,2023-08-25 17:04:39.585600+00:00,1,152,16


In [4]:
data = pd.read_csv('./messageTimeSeries.csv', header=0, index_col=0)
message = pd.DataFrame(data)

message = message.drop(columns=['start', 'end'])
message.head(10)

,_time,type,identifier,experiment
0,2023-08-25 17:04:39.427790+00:00,11.0,1,152
1,2023-08-25 17:04:39.427797+00:00,11.0,1,152
2,2023-08-25 17:04:39.427811+00:00,11.0,1,152
3,2023-08-25 17:04:39.427817+00:00,11.0,1,152
4,2023-08-25 17:04:39.427821+00:00,11.0,1,152
5,2023-08-25 17:04:39.427825+00:00,11.0,1,152
6,2023-08-25 17:04:39.427830+00:00,11.0,1,152
7,2023-08-25 17:04:39.427834+00:00,11.0,1,152
8,2023-08-25 17:04:39.427845+00:00,11.0,1,152
9,2023-08-25 17:04:39.427860+00:00,11.0,1,152


In [5]:
df = pd.concat([message, gossip])


del message
del gossip
gc.collect()

0

In [6]:
df.tail(10)

,_time,type,identifier,experiment
77,2023-09-14 23:09:59.801945+00:00,17.0,178,376
78,2023-09-14 23:09:59.802216+00:00,17.0,178,376
79,2023-09-14 23:09:59.802404+00:00,17.0,178,376
80,2023-09-14 23:09:59.802527+00:00,17.0,178,376
81,2023-09-14 23:09:59.802584+00:00,17.0,178,376
82,2023-09-14 23:09:59.802772+00:00,17.0,178,376
83,2023-09-14 23:09:59.803019+00:00,17.0,178,376
0,2023-09-14 23:43:30.722354+00:00,17.0,178,377
1,2023-09-14 23:43:30.722359+00:00,17.0,178,377
2,2023-09-14 23:43:30.723655+00:00,17.0,178,377


In [13]:
dfNoIndex = df.reset_index()
dfNoIndex["_time"] = pd.to_datetime(dfNoIndex["_time"],format="mixed", infer_datetime_format=True)
# dfNoIndex.head(10)

by_time = dfNoIndex.groupby([dfNoIndex['identifier'],dfNoIndex['experiment'],dfNoIndex['type'],pd.Grouper(key="_time", freq='10s')])['index'].count().reset_index()
dfAggTime = by_time.rename(columns={'index': "count"})

dfAggTime.tail(10)

,identifier,experiment,type,_time,count
72208,178,378,3.0,2023-09-15 00:11:50+00:00,42
72209,178,378,3.0,2023-09-15 00:12:00+00:00,42
72210,178,378,3.0,2023-09-15 00:12:10+00:00,56
72211,178,378,3.0,2023-09-15 00:12:20+00:00,42
72212,178,378,3.0,2023-09-15 00:12:30+00:00,42
72213,178,378,3.0,2023-09-15 00:12:40+00:00,56
72214,178,378,3.0,2023-09-15 00:12:50+00:00,42
72215,178,378,3.0,2023-09-15 00:13:00+00:00,42
72216,178,378,3.0,2023-09-15 00:13:10+00:00,56
72217,178,378,3.0,2023-09-15 00:13:20+00:00,28


In [15]:
dfAggTime.to_csv('./timeSeries10s.csv')

In [17]:
by_time2 = dfNoIndex.groupby([dfNoIndex['identifier'],dfNoIndex['experiment'],dfNoIndex['type'],pd.Grouper(key="_time", freq='1s')])['index'].count().reset_index()
dfAggTime = by_time2.rename(columns={'index': "count"})

dfAggTime.to_csv('./timeSeries.csv')
dfAggTime.tail(10)


,identifier,experiment,type,_time,count
537772,178,378,3.0,2023-09-15 00:12:58+00:00,14
537773,178,378,3.0,2023-09-15 00:13:01+00:00,14
537774,178,378,3.0,2023-09-15 00:13:04+00:00,14
537775,178,378,3.0,2023-09-15 00:13:07+00:00,14
537776,178,378,3.0,2023-09-15 00:13:10+00:00,14
537777,178,378,3.0,2023-09-15 00:13:13+00:00,14
537778,178,378,3.0,2023-09-15 00:13:16+00:00,14
537779,178,378,3.0,2023-09-15 00:13:19+00:00,14
537780,178,378,3.0,2023-09-15 00:13:22+00:00,14
537781,178,378,3.0,2023-09-15 00:13:25+00:00,14
